In [1]:
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

2022-12-05 14:24:38.465512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 14:24:38.577030: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 14:24:38.577043: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-05 14:24:38.595136: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-05 14:24:39.042479: W tensorflow/stream_executor/platform/de

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
import math as mt
import random as rd
from numpy.lib.arraysetops import intersect1d

# packages and modules from ml article linked here: https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# ignore some of above, trying a different model!
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

# for classifier 
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# other
from utils import type_to_int, type_to_int_dict_inv

In [3]:
# url='https://drive.google.com/file/d/1VzeIrzna8SDkJCnIrCIxn5LXnpx3WcrA/view?usp=sharing'
# url='https://drive.google.com/uc?id=' + url.split('/')[-2]
# poke = pd.read_csv(url) #import Pokemon data
poke = pd.read_csv("pokemon_clean.csv") #import Pokemon data

# url2='https://drive.google.com/file/d/1WR9i10kZ6bkI_7WV4fWmOXXNlaKgnBZ0/view?usp=share_link'
# url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
# training_data = pd.read_csv(url2) #import training data
training_data = pd.read_csv("trainingdata2202.csv") #import training data

poke = poke[poke.is_legendary == 0] #clean data for no legendaries
poke.reset_index(drop = True,inplace = True)
poke = poke.drop([131, 195, 196, 229, 348, 717]) #clean out problem children (Ditto, etc)
poke.reset_index(drop = True,inplace = True)

In [4]:
# zeroth row data point
# pokemon_name = training_data.iloc[0][1] #pokemon 1 of our team is index 1 for column
# pokemon_name_last = training_data.iloc[0][11] #last pokemon of opposing team is index 11 for column

# get desired stats
# temp = poke.loc[poke["name"]==pokemon_name].squeeze()
# temp["name"],temp["hp"],[temp["type1"],temp["type2"]],temp["attack"],temp["defense"],temp["sp_attack"],temp["sp_defense"],temp["speed"]

training_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,Trevenant,Grimer,Sawk,Zorua,Tangela,Meowstic,Vulpix,Forretress,Wishiwashi,...,Hydreigon,Hypno,Delphox,Bisharp,Blastoise,Empoleon,Jolteon,Oranguru,Slowking,Kangaskhan
1,1,Arcanine,Tyrantrum,Venusaur,Budew,Kangaskhan,Braixen,Lickilicky,Victreebel,Espeon,...,Sneasel,Weavile,Greninja,Scizor,Slaking,Minior,Goodra,Ribombee,Lucario,Aerodactyl
2,2,Sawk,Butterfree,Poliwhirl,Tynamo,Seaking,Floette,Quagsire,Litten,Cloyster,...,Silvally,Seviper,Alakazam,Swalot,Oricorio,Delphox,Nidoqueen,Salazzle,Steelix,Typhlosion
3,3,Butterfree,Wurmple,Kangaskhan,Vibrava,Corsola,Toucannon,Hitmonlee,Ursaring,Karrablast,...,Greninja,Alakazam,Manectric,Jellicent,Absol,Mienshao,Porygon-Z,Arcanine,Aegislash,Chimecho
4,4,Elgyem,Mamoswine,Ambipom,Greninja,Caterpie,Toucannon,Magcargo,Stunky,Torracat,...,Purrloin,Liepard,Pansage,Simisage,Pansear,Simisear,Servine,Phione,Froslass,Gallade


In [5]:
X = training_data.iloc[:,1:12]
Y = training_data.iloc[:,12:-1 ]

In [6]:
pokemon_features_df = pd.read_csv('pokemon_features.csv', index_col='name')
type_cols = pokemon_features_df.columns[:18]
stat_cols = pokemon_features_df.columns[18:]
typesX = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
statsX = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
typesY = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)
statsY = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)

print(typesX[0], statsX[0], '\n')
print(typesY[0], statsY[0])

[0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0] [474 166 167 325 105 130 465 210 150 330 145  80 435 160 180 466 187 155
 299 115 103 465 130 215 620 170 180 535 220 165 355 131 122] 

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0] [600 325 160]


In [7]:
# don't need this right now, it's for a single output binary classifier (from previous reference article) TO DELETE
# X_train, X_test, y_train, y_test = train_test_split(featureX.transpose(), featureY.transpose(), test_size=0.30, random_state=40)
# print(X_train.shape); print(X_test.shape)

For our type output, this could be a binary classifer with classes 1 (is the type) and 0 (is not the type) with 18 outputs. 

For the stats predictor, we cannot use a classifier unless we had all possible classes (every integer option: not feasible), instead we should use a regression model?)

Idea: two different models, one for classifying on type and one for regression

Idea2: use entirely a regression model and pick the type 1/2 that are the closest to 1

In [8]:
# regr = MultiOutputRegressor(Ridge(random_state=123)).fit(featureX, featureY)
# regr.predict(featureX[[0]])

This worked kinda but also not! Stats seems like reasonable numbers for stats, but type doesn't make much sense. Try: two models, one for type (classifier multioutput) and one for stats (regressor multioutput) and then use both of these results and combine them for our total output :)

In [9]:
def gotta_match_it_all(types_to_be_matched, stats_to_be_matched):
    top_num = 5
    types_matched = list(pokemon_features_df.index[((pokemon_features_df[type_cols] == 1) & (pokemon_features_df[type_cols] == types_to_be_matched)).any(axis=1)])
    pkm_matched_all = (pokemon_features_df[stat_cols] - stats_to_be_matched).abs().sum(axis=1).sort_values()
    if (len(types_matched) != 0):
        pkm_matched_with_type = pkm_matched_all[types_matched].sort_values()
        return pkm_matched_with_type.head(top_num)
    else:
        return pkm_matched_all.head(top_num)

In [10]:
def compare_results(ground_truth, pred_types, pred_stats, start_compare, end_compare, test_index):
    test_size = len(test_index)
    pred = [gotta_match_it_all(pred_types[i], pred_stats[i]).index for i in range(test_size)]
    truth_to_compare = ground_truth.iloc[:,start_compare:end_compare]
    accurate = [intersect1d(pred[i], truth_to_compare.iloc[test_index[i]]) for i in range(test_size)]
    return np.average([len(accurate[i]) for i in range(test_size)]) #/(end_compare-start_compare)

In [13]:
data_total = len(Y)
test_size = mt.floor(data_total/6)
test_index = sorted(rd.sample(range(0, data_total), test_size))
training_index = list(set(range(data_total)) - set(test_index))
print(len(test_index),len(training_index))

typesX_test = typesX[test_index]
statsX_test = statsX[test_index]
typesX_train = typesX[training_index]
statsX_train = statsX[training_index]
typesY_train = typesY[training_index]
statsY_train = statsY[training_index]

clf = MultiOutputClassifier(LogisticRegression()).fit(typesX_train, typesY_train)
typesPredict = clf.predict(typesX_test)
regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX_train, statsY_train)
statsPredict = regr.predict(statsX_test)

367 1835


In [19]:
num_val = 10
performance = np.zeros(num_val)

for i in range(num_val):
    data_total = len(Y)
    test_size = mt.floor(data_total/6)
    test_index = sorted(rd.sample(range(0, data_total), test_size))
    training_index = list(set(range(data_total)) - set(test_index))

    typesX_test = typesX[test_index]
    statsX_test = statsX[test_index]
    typesX_train = typesX[training_index]
    statsX_train = statsX[training_index]
    typesY_train = typesY[training_index]
    statsY_train = statsY[training_index]

    clf = MultiOutputClassifier(LogisticRegression()).fit(typesX_train, typesY_train)
    typesPredict = clf.predict(typesX_test)
    regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX_train, statsY_train)
    statsPredict=regr.predict(statsX_test)

    performance[i] =  compare_results(Y, typesPredict, statsPredict, 0, 10, test_index)

In [20]:
print(performance, '\n')
print("average intersection: ", np.mean(performance))

[0.92643052 0.95912807 0.88555858 0.94277929 0.96457766 0.86648501
 0.85013624 0.98365123 0.97820163 0.96457766] 

average intersection:  0.9321525885558583


In [15]:
# just to demonstrate how the model works 
gotta_match_it_all(clf.predict(tf.reshape(typesX[150],(1, -1))), regr.predict(tf.reshape(statsX[150],(1, -1))))

2022-12-05 14:25:25.623500: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-12-05 14:25:25.623543: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: papaya
2022-12-05 14:25:25.623552: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: papaya
2022-12-05 14:25:25.623645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2022-12-05 14:25:25.623672: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-12-05 14:25:25.623679: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.141.3 does not match DSO version 470.161.3 -- cannot find working devices in this configuration
2022-12-05 14:25:25.625804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow

name
Hydreigon     20.535621
Dragonite     24.437896
Kangaskhan    25.632259
Scizor        32.437896
Pinsir        39.367741
dtype: float64

In [16]:
gotta_match_it_all(clf.predict(tf.reshape(typesX[600],(1, -1))), regr.predict(tf.reshape(statsX[600],(1, -1))))

name
Ampharos      26.593432
Scizor        26.794777
Kangaskhan    31.794777
Hydreigon     38.615020
Dragonite     38.615020
dtype: float64

In [17]:
gotta_match_it_all(clf.predict(tf.reshape(typesX[1220],(1, -1))), regr.predict(tf.reshape(statsX[1220],(1, -1))))

name
Dragonite     70.703412
Hydreigon     70.703412
Kommo-o       81.214729
Garchomp     109.296588
Altaria      110.703412
dtype: float64

In [18]:
gotta_match_it_all(clf.predict(tf.reshape(typesX[1000],(1, -1))), regr.predict(tf.reshape(statsX[1000],(1, -1))))

name
Kangaskhan    21.334820
Hydreigon     22.029046
Dragonite     22.029046
Scizor        25.529247
Heracross     37.165382
dtype: float64

In [ ]:
# def type_match(types_to_be_matched):
#     pkm_matched = list(pokemon_features_df.index[(pokemon_features_df[type_cols] == types_to_be_matched).all(axis=1)])
#     if (len(pkm_matched) == 0):
#         pkm_matched = list(pokemon_features_df.index[((pokemon_features_df[type_cols] == 1) & (pokemon_features_df[type_cols] == types_to_be_matched)).any(axis=1)])
#     return pkm_matched

In [ ]:
# def stat_match(types_matched, stats_to_be_matched):
#     pkm_matched_all = (pokemon_features_df[stat_cols] - stats_to_be_matched).abs().sum(axis=1).sort_values()
#     if (len(types_matched) != 0):
#         pkm_matched_with_type = pkm_matched_all[types_matched].sort_values()
#     return pkm_matched_all.head(5), pkm_matched_with_type.head(5)

In [ ]:
### test cell
# pd.Series(typesPredict.sum(axis=1)).value_counts()
# np.where(typesPredict.sum(axis=1)==1)
# print(type_match(typesPredict[0]))
# stat_match(type_match(typesPredict[0]), statsPredict[0])

In [ ]:
# def compare_results(ground_truth,prediction_types,prediction_stats,start_compare,end_compare,start_index,end_index):
#   percent_accurate = np.zeros(end_index-start_index)
#   for i in range(start_index,end_index):
#     predict = gotta_match_it_all(prediction_types[i], prediction_stats[i])
#     percent_accurate[i] = len(intersect1d(predict.index,ground_truth.iloc[i][start_compare:end_compare]))/(end_compare-start_compare)
#   return np.average(percent_accurate)

In [ ]:
# plot_data = np.zeros(19)
# for i in range(1,20):
#   plot_data[i-1] = compare_results(Y.iloc([test_index]),typesPredict,statsPredict,0,i,0,test_size)
# plt.plot(range(1,20),plot_data)

In [ ]:
### test_cell
# pred_1 = [gotta_match_it_all(typesPredict[i], statsPredict[i]).index for i in range(test_size)]
# truth_to_compare_1 = Y.iloc[:,0:1]
# accurate_1 = [intersect1d(pred_1[i], truth_to_compare_1.iloc[test_index[i]]) for i in range(test_size)]

# compare_results(Y, typesPredict, statsPredict, 0, 20, test_index)
# #[gotta_match_it_all(typesPredict[i], statsPredict[i]).index for i in range(test_size)]
# #Y.iloc[:,1:5]
# #[len(accurate_1[i]) for i in range(test_size)]

# plot_data = [compare_results(Y, typesPredict, statsPredict, i, i+1, test_index) for i in range(20)]
# plt.plot(range(20),plot_data)

This binary multiclass output does not return all outputs as having a type. We really need it to return 1 or 2 types for our output pokemon (this is a BIG part of choosing pokemon to battle). New idea: make each type a class with an int 0-17, or 1-18, and maybe another class as a placeholder for a type when a pokemon has only 1 type (or simply have [18 18]. Then, use a multiclass classifier with an output of 2 (representing types)

Need to make data changes :'(

In [22]:
# types_pred = clf.predict(typesX)
# types_pred_nonzero_indecies = np.where(np.sum(types_pred,axis=1) >= 4)[0]
# types_pred_nonzero = types_pred[types_pred_nonzero_indeces]

# types_pred_nonzero_indecies

MAYBE types doooooo only matter in 1/3 of our battles ?? (:

If type isn't  all zero, then condition on type and search for nearest neighbor of that type?
weight based on the team's deficiency


CHAMPIONSHIP TEAMS WITH USER INPUT

In [23]:
# Championship Teams
championship_teams = np.array([["Milotic","Gothitelle","Volcarona","Escavalier","Conkeldurr","Hydreigon"],
                      ["Milotic","Aerodactyl","Scrafty","Eelektross","Krookodile","Haxorus"],
                      ["Milotic","Cofagrigus","Scrafty","Reuniclus","Conkeldurr","Emboar"],
                      ["Salamence","Electivire","Incineroar","Gastrodon","Snorlax","Ferrothorn"],
                      ["Gengar","Sceptile","Gliscor","Incineroar","Haxorus","Politoed"],
                      ["Metagross","Electivire","Gliscor","Incineroar","Mimikyu","Snorlax"],
                      ["Gyarados","Garchomp","Talonflame","Pachirisu","Gothitelle","Gardevoir"],
                      ["Charizard","Lucario","Garchomp","Mamoswine","Tyranitar","Salamence"],
                      ["Mawile","Conkeldurr","Eelektross","Hydreigon","Tyranitar","Aegislash"],
                      ["Altaria","Gardevoir","Marowak","Krookodile","Whimsicott","Skarmory"],
                      ["Espeon", "Arcanine", "Garchomp", "Ninetales", "Mandibuzz", "Ampharos"],
                      ["Kingdra","Gyarados","Marowak","Garchomp","Oranguru","Ferrothorn"]])
user_input = np.array(["Gyarados","Raichu","Quagsire","Cloyster","Slowking"])
user_input = np.reshape(np.tile(user_input,[len(championship_teams)]),[len(championship_teams),len(user_input)])
combined  = np.concatenate((user_input,championship_teams),axis = 1)
pdCombined = pd.DataFrame(combined,columns =[0,1,2,3,4,5,6,7,8,9,10])
pdCombined


,0,1,2,3,4,5,6,7,8,9,10
0,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Gothitelle,Volcarona,Escavalier,Conkeldurr,Hydreigon
1,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Aerodactyl,Scrafty,Eelektross,Krookodile,Haxorus
2,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Cofagrigus,Scrafty,Reuniclus,Conkeldurr,Emboar
3,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Salamence,Electivire,Incineroar,Gastrodon,Snorlax,Ferrothorn
4,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Gengar,Sceptile,Gliscor,Incineroar,Haxorus,Politoed
5,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Metagross,Electivire,Gliscor,Incineroar,Mimikyu,Snorlax
6,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Gyarados,Garchomp,Talonflame,Pachirisu,Gothitelle,Gardevoir
7,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Charizard,Lucario,Garchomp,Mamoswine,Tyranitar,Salamence
8,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Mawile,Conkeldurr,Eelektross,Hydreigon,Tyranitar,Aegislash
9,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Altaria,Gardevoir,Marowak,Krookodile,Whimsicott,Skarmory


In [24]:
typesXuser = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in pdCombined.iterrows()], axis=0)
statsXuser = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in pdCombined.iterrows()], axis=0)
typesPredictUser = clf.predict(typesXuser)
statsPredictUser = regr.predict(statsXuser)

best_mons = np.empty(60, dtype="<U12")
for i in range(12):
  for j in range(5):
    best_mons[i+j]=gotta_match_it_all(typesPredictUser[i],statsPredictUser[i]).index[j]
best_mons

array(['Garchomp', 'Salamence', 'Charizard', 'Gyarados', 'Salamence',
       'Tyranitar', 'Salamence', 'Salamence', 'Salamence', 'Charizard',
       'Salamence', 'Garchomp', 'Charizard', 'Salamence', 'Blaziken',
       'Houndoom', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', ''], dtype='<U12')

In [25]:
gotta_match_it_all(typesPredictUser[1],statsPredictUser[1]).index[0]
best_mons = np.empty(60, dtype="<U12")
best_mons[1] = "Salamence"

best_mons

array(['', 'Salamence', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', ''], dtype='<U12')